## Exercício 1

Siga o tutorial do site [Physics-informed Neural Networks: a simple tutorial with PyTorch](https://medium.com/@theo.wolf/physics-informed-neural-networks-a-simple-tutorial-with-pytorch-f28a890b874a). Neste exemplo, você irá resolver a EDO do resfriamento de uma caneca de café em etapas:

1. Resolva analiticamente a EDO do resfriamento de uma caneca de café. A equação é dada por:

```math
   \frac{dT}{dt} = r(T_{amb} - T)
```
onde $T$ é a temperatura do café, $T_{amb} = 25$ ºC é a temperatura ambiente e $r = 0.005$ 1/s é uma taxa de resfriamento.

2. Resolva a EDO usando o método de Runge-Kutta de quarta ordem (RK4) e compare com a solução analítica. Use o comando `scipy.integrate.solve_ivp`.

3. Usando a solução analítica, ou RK4, gere dados sintéticos para treinar a PINN. Cerca de 10 pontos no intervalo de 0 a 200 segundos (veja exemplo no tutorial). Some um ruído gaussiano com média 0 e desvio padrão 0.5 em cada ponto.

4. Tente usar uma NN de regressão simples para ajustar os dados sintéticos e extrapolar para tempos maiores (até 1000 segundos). Compare com a solução analítica. Sabemos que a extrapolação será péssima.

5. Agora, siga o tutorial e implemente a PINN incluindo as restrições físicas na minimização da perda, mas assumindo que conhecemos o valor da taxa $r = 0.005$ 1/s. Compare com a solução analítica e com a NN de regressão simples.

6. Ainda seguindo o tutorial, implemente a PINN sem conhecer o valor da taxa $r$. A rede deve ser capaz de descobrir o valor correto. Compare com a solução analítica e com a NN de regressão simples.

In [1]:
import numpy as np
import tensorflow as tf

In [3]:
def grad(outputs, inputs):
    """Computes the partial derivative of an output with respect to an input."""
    with tf.GradientTape() as tape:
        tape.watch(inputs)
        grads = tape.gradient(outputs, inputs)
    return grads

def physics_loss(model, R, Tenv):
    """The physics loss of the model."""
    # Create collocation points
    ts = tf.linspace(0, 1000, num=1000)[:, tf.newaxis]  # Shape (1000, 1)
    ts = tf.Variable(ts, trainable=True)  # Ensure `ts` is differentiable

    # Run the collocation points through the network
    temps = model(ts)

    # Get the gradient
    with tf.GradientTape() as tape:
        tape.watch(ts)
        dT = tape.gradient(temps, ts)

    # Compute the ODE
    ode = dT - R * (Tenv - temps)

    # MSE of ODE
    return tf.reduce_mean(tf.square(ode))

In [5]:
class Net(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(Net, self).__init__(*args, **kwargs)
        # Make r a trainable variable
        self.r = tf.Variable(0.0, trainable=True, dtype=tf.float32)

    def call(self, inputs):
        # Define the forward pass (to be implemented based on your architecture)
        pass

def physics_loss_discovery(model, Tenv):
    """Physics loss for discovering the parameter r."""
    # Create collocation points
    ts = tf.linspace(0, 1000, num=1000)[:, tf.newaxis]  # Shape (1000, 1)
    ts = tf.Variable(ts, trainable=True)  # Ensure `ts` is differentiable

    # Run the collocation points through the network
    temps = model(ts)

    # Compute the gradient of temps with respect to ts
    with tf.GradientTape() as tape:
        tape.watch(ts)
        dT = tape.gradient(temps, ts)

    # Use the differentiable parameter r
    pde = model.r * (Tenv - temps) - dT

    # Return the mean squared error of the PDE residual
    return tf.reduce_mean(tf.square(pde))